In [ ]:
# importamos wget y las librerías que nos faltan
from wget import download
from os import path
import pandas as pd
#https://github.com/agustinbx/DataSet-Trabajo/winequality_BR.csv



# descargamos la base de datos
if not path.exists("winequality_BR.csv"):
  download("https://github.com/agustinbx/DataSet-Trabajo/raw/main/winequality_BR.csv")
else:
  print("No vamos a bajar el archivo de itunes porque ya existe!")

In [ ]:
raw_ds = pd.read_csv("winequality_BR.csv", delimiter=';')
raw_ds

In [ ]:
raw_ds.columns.to_list()

In [27]:
raw_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3232 entries, 0 to 3231
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  3232 non-null   object 
 1   fixed acidity         3232 non-null   float64
 2   volatile acidity      3232 non-null   float64
 3   citric acid           3232 non-null   float64
 4   residual sugar        3232 non-null   float64
 5   chlorides             3232 non-null   float64
 6   free sulfur dioxide   3232 non-null   float64
 7   total sulfur dioxide  3232 non-null   float64
 8   density               3232 non-null   float64
 9   pH                    3232 non-null   float64
 10  sulphates             3232 non-null   float64
 11  alcohol               3232 non-null   object 
 12  quality               3232 non-null   int64  
 13  Unnamed: 13           0 non-null      float64
 14  Unnamed: 14           0 non-null      float64
 15  Unnamed: 15          

type: tipo de uva con la que se elabora el vino.

fixed acidity: cantidad de ácidos no volátiles presentes en el vino, medida en gramos por litro.

volatile acidity: cantidad de ácidos volátiles presentes en el vino, medida en gramos por litro. 

citric acid: contenido de ácido cítrico en el vino, medido en gramos por litro.

residual sugar: cantidad de azúcar que queda en el vino después de la fermentación, medida en gramos por litro. 

chlorides: concentración de cloruros (sales) en el vino, medida en gramos por litro.

free sulfur dioxide: cantidad de dióxido de azufre que no está ligado químicamente en el vino, medida en miligramos por litro. 

total sulfur dioxide: suma del dióxido de azufre libre y el combinado en el vino, medida en miligramos por litro.

density: medida de la masa por unidad de volumen del vino, utilizada para estimar la concentración de sólidos disueltos, medida en gramos por centímetro cúbico.

pH: medida de la acidez o alcalinidad del vino. 

sulphates: concentración de sales de sulfato en el vino, medida en gramos por litro.

alcohol: contenido alcohólico del vino, medido en porcentaje de volumen (% vol).

quality: puntuación del vino, con una escala que va de 0 a 10.


In [ ]:
import pandas_profiling as pp

#generamos el reporte
report = pp.ProfileReport(raw_ds)

# lo mostramos interactivo en nuestra notebook
#report.to_notebook_iframe()

#descarga el archivo html (lo abrimos desde google)
report.to_file(output_file="Profile_report_winequality.html")



La correlación es un concepto estadístico que indica la dirección y la fuerza de la relación entre dos variables. Cuando dos variables están correlacionadas, significa que los cambios en una de ellas están asociados con cambios en la otra. Esta asociación puede ser positiva (cuando una variable aumenta, la otra también lo hace) o negativa (cuando una variable aumenta, la otra disminuye).

El coeficiente de correlación de Pearson es la medida más utilizada para cuantificar esta relación. Se representa con la letra  r  y toma valores entre -1 y 1. Un valor de  r  cercano a 1 indica una fuerte correlación positiva, mientras que un valor cercano a -1 indica una fuerte correlación negativa. Un valor cercano a 0 sugiere que no hay una correlación lineal evidente entre las variables.

Correlación no implica causalidad. Dos variables pueden estar correlacionadas sin que una cause necesariamente la otra.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generamos la matriz de correlacion, teniendo en cuenta unicamente valores numericos
numeric_data = raw_ds.select_dtypes(include=[np.number])
#numeric_data = numeric_data.dropna()
correlation_matrix = numeric_data.corr()

correlation_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
#imprimimos la matriz como un heatmap
plt.figure(figsize=(16,12))
sns.heatmap(correlation_matrix, vmin=-1.0, vmax=1.0, center=0.0, annot=True, cmap='coolwarm')
plt.title('Heatmap de la Matriz de Correlación', fontsize=18)
plt.show()

Una regla de esas que no tienen demostración pero que funciona bastante bien (el término en inglés para definirlas es "rules of thumb") dice que si la correlación da por encima de 0.7 (o por debajo de -0.7), la relación entre esas dos variables merece un poco de análisis extra.

In [ ]:
# tomamos el valor absoluto de las correlaciones, umbralamos las mayores a 0.7
correlation_matrix_umbralizada = correlation_matrix.abs() > 0.7 

# aprovechamos y sacamos la diagonal
np.fill_diagonal(correlation_matrix_umbralizada.values, 0)

# e imprimimos la matriz como un heatmap
plt.figure(figsize=(16,12))
sns.heatmap(correlation_matrix_umbralizada, vmin=0.0, vmax=1.0, center=0.0, annot=True, cmap= 'coolwarm')
plt.show()

In [ ]:
# iteramos por cada una de las columnas
for i, col in enumerate(correlation_matrix_umbralizada.columns):
  # y por cada una de las filas (para eso usamos el índice)
  for j in range(i,len(correlation_matrix_umbralizada.index)):
    # extraemos el nombre de la fila en la que estamos
    row = correlation_matrix_umbralizada.index[j]
    # si en esa posición tenemos un True en la matriz umbralizada
    if correlation_matrix_umbralizada.loc[row, col]:
      # nos traemos el valor de esa correlación
      correlation = correlation_matrix.loc[row, col]
      # representamos en un scatter plot cuadrado los valores de ambas variables
      plt.figure(figsize=(6, 6))
      plt.scatter(raw_ds[row], raw_ds[col])
      # y lo mostramos bonito
      plt.title(f"Scatter plot entre {col} y {row} - r = {correlation:.4f}")
      plt.xlabel(row)
      plt.ylabel(col)
      plt.grid(True)
      plt.show()    